## Day 24 - Colliding hailstones

**Part 1: Disregarding z-axis, how many hailstones will collide between with an X and Y position each at least 200000000000000 and at most 400000000000000.**

In [115]:
with open("./example.txt") as f:
    example_lines = [line.strip() for line in f.readlines()]

with open("./input.txt") as f:
    input_lines = [line.strip() for line in f.readlines()]

example_lines

['19, 13, 30 @ -2,  1, -2',
 '18, 19, 22 @ -1, -1, -2',
 '20, 25, 34 @ -2, -2, -4',
 '12, 31, 28 @ -1, -2, -1',
 '20, 19, 15 @  1, -5, -3']

In [116]:
def intersection_time2(x1, x2, y1, y2, vx1, vx2, vy1, vy2) -> float:
    return (x2 - x1 + (vx1/vy1)*(y1-y2))/((vx1*vy2)/vy1 - vx2)

In [117]:
def position_at_time2(x2, y2, vx2, vy2, t2):
    return (x2 + vx2*t2, y2 + vy2*t2)

In [118]:
def t1_from_xval1(xs, vx1, xval1):
    return (xval1 - xs)/vx1

In [119]:
from itertools import combinations

def part1(lines: list[str], example: bool = False) -> int:
    lower_bound = 7 if example else 200000000000000
    upper_bound = 27 if example else 400000000000000

    hailstones = [
        tuple(map(int,
                  (line.split(",")[0], line.split(", ")[1], line.split("@ ")[-1].split(",")[0], line.split("@ ")[-1].split(", ")[1])
                  ))
        for line in lines
    ]  # (X, Y, vX, vY)

    found = 0

    for (x1, y1, vx1, vy1), (x2, y2, vx2, vy2) in combinations(hailstones, r=2):
        try:
            t2 = intersection_time2(x1, x2, y1, y2, vx1, vx2, vy1, vy2)
            xval2, yval2 = position_at_time2(x2, y2, vx2, vy2, t2)
            t1 = t1_from_xval1(x1, vx1, xval2)

        except:
            continue

        if (
            lower_bound <= xval2 <= upper_bound
            and lower_bound <= yval2 <= upper_bound
            and t2 >= 0
            and t1 >= 0
        ):
            found += 1

    return found

assert part1(example_lines, example=True) == 2
part1(input_lines, example=False)  # with some Algebra help from Kate!

**Part 2: Throw a rock to hit all the hailstones, now including the z-axis....**

In [120]:
example_lines

['19, 13, 30 @ -2,  1, -2',
 '18, 19, 22 @ -1, -1, -2',
 '20, 25, 34 @ -2, -2, -4',
 '12, 31, 28 @ -1, -2, -1',
 '20, 19, 15 @  1, -5, -3']

In [121]:
# class Hailstone:
#     def __init__(self, sx: int, sy: int, sz: int, vx: int, vy: int, vz: int):
#         self.sx: int = sx
#         self.sy: int = sy
#         self.sz: int = sz
#         self.vx: int = vx
#         self.vy: int = vy
#         self.vz: int = vz

#     def __repr__(self) -> str:
#         return f"H({self.sx}, {self.sy}, {self.sz} @ {self.vx}, {self.vy}, {self.vz})"

In [162]:
import sympy as sym
sym.init_printing()

# x,y,z,vx,vy,vz = sym.symbols("x,y,z,vx,vy,vz")
x,y,z,vx,vy,vz = 19, 13, 30, -2, 1, -2
x2,y2,z2,vx2,vy2,vz2 = 18,19,22,-1,-1,-2
x3,y3,z3,vx3,vy3,vz3 = 20,25,34,-2,-2,-4
X,Y,Z,VX,VY,VZ = sym.symbols("X Y Z VX VY VZ")
# X = sym.Symbol("X", positive=True)
# Y = sym.Symbol("Y", positive=True)
# Z = sym.Symbol("Z", positive=True)
# VX = sym.Symbol("VX", positive=True)
# VY = sym.Symbol("VY", positive=True)
# VZ = sym.Symbol("VZ", positive=True)
# t1 = sym.Symbol("t1", positive=True)
# t2 = sym.Symbol("t2", positive=True)
# t3 = sym.Symbol("t3", positive=True)
t1,t2,t3 = sym.symbols("t1 t2 t3")
eq_x = sym.Eq(x+vx*t1, X+VX*t1)
eq_y = sym.Eq(y+vy*t1, Y+VY*t1)
eq_z = sym.Eq(z+vz*t1, Z+VZ*t1)
eq_x2 = sym.Eq(x2+vx2*t2, X+VX*t2)
eq_y2 = sym.Eq(x2+vy2*t2, Y+VY*t2)
eq_z2 = sym.Eq(x2+vz2*t2, Z+VZ*t2)
eq_x3 = sym.Eq(x3+vx3*t3, X+VX*t3)
eq_y3 = sym.Eq(x3+vy3*t3, Y+VY*t3)
eq_z3 = sym.Eq(x3+vz3*t3, Z+VZ*t3)

sym.solve([eq_x,eq_y,eq_z,eq_x2, eq_y2, eq_z2, eq_x3, eq_y3, eq_z3])  # not quite sure why this isn't right / doesn't work?

In [123]:
sym.solve([X+Y-5, Y*X+2])

In [124]:
def part2(lines: list[str]) -> int:
    """
    Returns the sum of the X, Y, Z, coordinates that this brutal rock would start from.
    """
    hailstones = [
        tuple(map(int, line.replace(" @", ",").split(", ")))
        for line in lines
    ]
    X,Y,Z,VX,VY,VZ = sym.symbols("X Y Z VX VY VZ")
    
    equations = []

    # can actually just throw all possible equations into sympy, but a bit slower!

    for i, (x, y, z, vx, vy, vz) in enumerate(hailstones):
        equations.append(
            (Y-y)*(vx-VX) - (X-x)*(vy-VY)
        )
        equations.append(
            (Z-z)*(vx-VX) - (X-x)*(vz-VZ)
        )
        if i < 2:
            continue
        answers = [soln for soln in sym.solve(equations)]
        if len(answers) == 1:
            break
    
    answer = answers.pop()

    print(answer)
    return sum((answer[X], answer[Y], answer[Z]))


assert part2(example_lines) == 47
part2(input_lines)

{VX: -3, VY: 1, VZ: 2, X: 24, Y: 13, Z: 10}
{VX: 26, VY: -331, VZ: 53, X: 270392223533307, Y: 463714142194110, Z: 273041846062208}


dangerous copying paste the sympy print above lol. Doubled it for whatever reason!

In [125]:
sym.init_printing(False)
part2(input_lines)

{VX: 26, VY: -331, VZ: 53, X: 270392223533307, Y: 463714142194110, Z: 273041846062208}


1007148211789625